In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

In [2]:
def clean_cols(columns):
    columns = columns.str.replace('<', '').str.replace('>', '')
    columns = columns.str.lower()
    return columns

In [3]:
company_names = ['amazon', 'apple', 'goog', 'googl', 'microsoft', 'tesla']
stocks = pd.DataFrame()

for company in company_names:
    stocks_company = pd.read_csv(f'stocks/{company}_stocks.csv')
    stocks_company.columns = clean_cols(stocks_company.columns)
    stocks = pd.concat([stocks, stocks_company])
stocks['date'] = stocks['date'].astype(str)
stocks['time'] = stocks['time'].astype(str)
stocks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63373 entries, 0 to 10563
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ticker  63373 non-null  object 
 1   per     63373 non-null  int64  
 2   date    63373 non-null  object 
 3   time    63373 non-null  object 
 4   open    63373 non-null  float64
 5   high    63373 non-null  float64
 6   low     63373 non-null  float64
 7   close   63373 non-null  float64
 8   vol     63373 non-null  int64  
dtypes: float64(4), int64(2), object(3)
memory usage: 4.8+ MB


In [4]:
stocks['ticker'] = stocks['ticker'].apply(lambda x: x.split('.')[1])
stocks.loc[stocks['time'] == '0', ['time']] = '000000'
stocks = stocks[stocks['time'] != '10000']
stocks.drop(['per'], axis=1, inplace=True)

In [5]:
stocks['time'].value_counts().sort_index()

000000    3096
170000    5965
180000    9055
190000    9058
200000    9058
210000    9058
220000    9045
230000    9031
Name: time, dtype: int64

In [6]:
stocks.head()

,ticker,date,time,open,high,low,close,vol
0,AMZN,20150102,180000,312.58,314.75,312.1100,313.52,404201
1,AMZN,20150102,190000,313.25,313.59,308.6300,308.96,509208
2,AMZN,20150102,200000,308.84,310.12,306.9601,309.75,427854
3,AMZN,20150102,210000,309.77,309.93,308.0500,308.40,209933
4,AMZN,20150102,220000,308.41,308.48,307.2500,308.35,158209


In [7]:
stocks['date_'] = stocks['date'] + stocks['time']
stocks['date_'] = pd.to_datetime(stocks['date_'], format='%Y%m%d%H%M%S')

stocks.drop(['date', 'time'], axis=1, inplace=True)

In [8]:
#match tweets timezone
stocks['date_'] = stocks['date_'] - timedelta(hours=1)

In [9]:
def get_hours(col):
    return col['date_'].hour

In [10]:
stocks['time'] = stocks.apply(get_hours, axis=1)
stocks.time.value_counts().sort_index()

16    5965
17    9055
18    9058
19    9058
20    9058
21    9045
22    9031
23    3096
Name: time, dtype: int64

In [11]:
stocks.drop(['time'], axis=1, inplace=True)

In [12]:
cols = list(stocks.columns[:-1])
cols.insert(0, 'date_')
stocks = stocks[cols]

In [13]:
stocks.head()

,date_,ticker,open,high,low,close,vol
0,2015-01-02 17:00:00,AMZN,312.58,314.75,312.1100,313.52,404201
1,2015-01-02 18:00:00,AMZN,313.25,313.59,308.6300,308.96,509208
2,2015-01-02 19:00:00,AMZN,308.84,310.12,306.9601,309.75,427854
3,2015-01-02 20:00:00,AMZN,309.77,309.93,308.0500,308.40,209933
4,2015-01-02 21:00:00,AMZN,308.41,308.48,307.2500,308.35,158209


In [14]:
for i in stocks['ticker'].unique():
    stocks[stocks['ticker'] == i].to_csv(f'stocks_clean/{i}.csv', index=None)